## Projet de carte interactive des resultats Législatives et Européenes

### Imports

In [1]:
import geopandas as gpd
import pandas as pd 
import folium
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup
from folium import plugins
from folium import Choropleth, LayerControl
import sys
sys.path.append('scripts')  # Add the scripts directory to the system path
from import_function import import_txt_file, process_and_merge_scores, process_and_merge_scores_eur

### Legislatives 1<sup>er</sup> tour 2022

#### Import csv https://www.data.gouv.fr/fr/datasets/elections-legislatives-des-12-et-19-juin-2022-resultats-definitifs-du-premier-tour/

In [2]:
file_path = "Source/2022/resultats-par-niveau-burvot-t1-france-entiere.txt"
Scores_Leg_t1_2022 = import_txt_file(file_path, 'Legislatives_2022', 1)

#### Transposing columns

In [3]:
Scores_Leg_t1_2022_2 = process_and_merge_scores(Scores_Leg_t1_2022)

### Européennes 2024

#### Import csv https://www.data.gouv.fr/fr/datasets/resultats-des-elections-europeennes-du-9-juin-2024/

In [4]:
file_path = "Source/2024/resultats-definitifs-par-bureau-de-vote.csv"
Scores_Eur_2024 = import_txt_file(file_path, 'Européennes_2024',1, encoding='utf-8')

#### Restriction Paris

In [5]:
Scores_Eur_2024_75 = Scores_Eur_2024[(Scores_Eur_2024['Code département'] == '75')]

#### Transposing columns

In [6]:
Scores_Eur_2024_75_2 = process_and_merge_scores_eur(Scores_Eur_2024_75)

In [8]:
#Scores_Eur_2024_752 = Scores_Eur_2024_75_2[Scores_Eur_2024_75_2['Code BV'].str.startswith(('16', '17'))].assign(
Scores_Eur_2024_752 = Scores_Eur_2024_75_2.assign(
    **{'Code BV': lambda df: df['Code BV'].apply(
        lambda x: f"{int(str(x)[:2])}-{int(str(x)[2:])}" if len(str(x)) == 4 else f"{int(str(x)[:1])}-{int(str(x)[2:])}"
    )}
).rename(columns={"Code BV": "id_bv"})

### Zoom Paris

In [9]:
Leg_t1_2022_75 = Scores_Leg_t1_2022_2[
    (Scores_Leg_t1_2022_2['Code du département'] == '75')
    #& (Scores_Leg_t1_2022['Code de la circonscription'].isin([3, 4])
].dropna(axis=1, how='all').assign(
    **{'Code du b.vote': lambda df: df['Code du b.vote'].apply(
        lambda x: f"{int(str(x)[:2])}-{int(str(x)[2:])}" if len(str(x)) == 4 else f"{int(str(x)[:1])}-{int(str(x)[2:])}"
    )}
).rename(columns={"Code du b.vote": "id_bv"})

In [23]:
BDV_Map = gpd.read_file("Source/secteurs-des-bureaux-de-vote.geojson")
BDV = pd.read_csv("Source/ListeBDVParisv5.csv")
BDV4 = BDV.drop('Unnamed: 0', axis=1)
BDV4 = BDV4.rename(columns={"Num": "num_bv", "Arrondissement": "arrondissement"})
BDV4b = BDV4.drop_duplicates(["arrondissement","num_bv","Circonscription"])
BDV4b = BDV4b.map(str)
BDV_Map_4 = pd.merge(BDV_Map,BDV4b,how='left',left_on=['arrondissement',"num_bv"],right_on=['arrondissement',"num_bv"])

In [10]:
BDV_Map_24 = gpd.read_file('Source/2024/elections-europeennes-2024.geojson')

In [11]:
BDV_Map_24_2 = BDV_Map_24[['geometry','id_bv']]

In [ ]:
BDV_Map_4[BDV_Map_4['arrondissement']=='4']

In [ ]:
BDV_Map_75=BDV_Map_4.dissolve(by='id_bv')[['geometry']]

### Euro 24

In [ ]:
Leg_t1_2022_75.columns

In [12]:
def transform_percentage(series):
    return pd.to_numeric(series.str.replace('%', '').str.replace(',', '.'), errors='coerce')

In [13]:
nuance_list=['LFI', 'LVEC', 'LENS', 'LCOM', 'LDVG', 'LUG']

# Apply the transformation to both '% Voix/exprimés' and '% Abstentions'
Scores_Eur_2024_752temp = Scores_Eur_2024_752[Scores_Eur_2024_752['Nuance liste'].isin(nuance_list)][['id_bv', '% Voix/exprimés', '% Abstentions']].assign(
    **{
        '% Voix/exprimés': lambda df: transform_percentage(df['% Voix/exprimés']),
        '% Abstentions': lambda df: transform_percentage(df['% Abstentions'])
    }
)
Scores_Eur_2024_752temp2 = Scores_Eur_2024_752temp.groupby('id_bv', as_index=False).agg({'% Voix/exprimés': 'sum', '% Abstentions': 'first'})
BDV4_Scores_map = pd.merge(BDV_Map_24_2,Scores_Eur_2024_752temp2,how='inner',left_on=['id_bv'],right_on=['id_bv'])

In [14]:
Leg_t1_2022_75 = (
    Scores_Leg_t1_2022_2[Scores_Leg_t1_2022_2['Code du département'] == '75']
    .dropna(axis=1, how='all')
    .assign(
        **{
            'Code du b.vote': lambda df: df['Code du b.vote'].apply(
        lambda x: f"{int(str(x)[:2])}-{int(str(x)[2:])}" if len(str(x)) == 4 else f"{int(str(x)[:1])}-{int(str(x)[2:])}"
        ),
            '% Voix/Exp': lambda df: transform_percentage(df['% Voix/Exp']),
            '% Abs/Ins': lambda df: transform_percentage(df['% Abs/Ins'])
        }
    )
    .rename(columns={"Code du b.vote": "id_bv"})
)

Leg_t1_2022_752=Leg_t1_2022_75[Leg_t1_2022_75['Nuance']=='NUP'][['id_bv', '% Voix/Exp','% Abs/Ins']]
BDV4_Scores_map2 = pd.merge(BDV4_Scores_map,Leg_t1_2022_752,how='inner',left_on=['id_bv'],right_on=['id_bv'])

In [15]:
BDV4_Scores_map2.head()

,geometry,id_bv,% Voix/exprimés,% Abstentions,% Voix/Exp,% Abs/Ins
0,"POLYGON ((2.34944 48.84065, 2.34941 48.84064, ...",5-6,72.79,35.41,49.00,36.24
1,"POLYGON ((2.35861 48.83894, 2.35851 48.83911, ...",5-19,68.38,34.49,35.93,37.87
2,"POLYGON ((2.32622 48.84686, 2.32623 48.84685, ...",6-17,62.97,34.59,26.65,41.37
3,"POLYGON ((2.31995 48.85495, 2.31971 48.85503, ...",7-1,56.76,37.84,41.33,43.68
4,"POLYGON ((2.31629 48.85878, 2.31625 48.85877, ...",7-3,52.74,39.28,11.01,47.55


In [16]:
BDV4_Scores_map2['Delta participation'] = (BDV4_Scores_map2['% Abs/Ins'] - BDV4_Scores_map2['% Abstentions'])
BDV4_Scores_map2['Delta Nupes'] = (BDV4_Scores_map2['% Voix/exprimés'] - BDV4_Scores_map2['% Voix/Exp'])
BDV4_Scores_map2['Multiplicateur'] = BDV4_Scores_map2['Delta Nupes']/BDV4_Scores_map2['Delta participation'] 

In [32]:
#BDV4_Scores_map2.to_csv('BDV4_Scores_map2.csv')
Scores_Leg_t1_2022_2['Code du b.vote'].to_csv('zf.csv')

In [18]:
# Initialize the map centered around a central point
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)  # Centered around Paris

# Define the function to add a layer to the map
def add_choropleth_layer(data, column, name, geojson):
    Choropleth(
        geo_data=geojson,
        data=data,
        columns=['id_bv', column],
        key_on='feature.properties.id_bv',
        fill_color='YlGnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=name,
        nan_fill_color='white'
    ).add_to(m)

# Convert GeoDataFrame to GeoJSON
geojson = BDV4_Scores_map2.to_json()

# Add layers to the map
add_choropleth_layer(BDV4_Scores_map2, '% Voix/exprimés', '% Voix/exprimés 24', geojson)
add_choropleth_layer(BDV4_Scores_map2, '% Voix/Exp', '% Voix/Exp 22', geojson)
add_choropleth_layer(BDV4_Scores_map2, 'Delta participation', 'Delta participation', geojson)
add_choropleth_layer(BDV4_Scores_map2, 'Delta Nupes', 'Delta Nupes', geojson)
add_choropleth_layer(BDV4_Scores_map2, 'Multiplicateur', 'Multiplicateur', geojson)

# Add layer control to the map
LayerControl().add_to(m)

m.save('map.html')

In [130]:
BDV4_Scores_map2.explore("% Voix/exprimés", cmap="Reds",fill_opacity=0.15)  

In [ ]:
gdf['abst'] = 1 -(gdf['nb_votant'] / gdf['nb_inscr'])

In [ ]:
gdf.explore('abst', cmap="Reds",fill_opacity=0.15,tooltip=['id_bv','abst'])  

### Legis 22

In [ ]:
Leg_t1_2022_75temp=Leg_t1_2022_75[Leg_t1_2022_75['Nuance']=='NUP'][['id_bv',"% Abs/Ins"]]
BDV4_Scores_map = pd.merge(BDV_Map_75,Leg_t1_2022_75temp,how='inner',left_on=['id_bv'],right_on=['id_bv'])

In [ ]:
m=BDV4_Scores_map.explore("% Abs/Ins", cmap="Blues",fill_opacity=0.15)  

In [ ]:
Leg_t1_2022_75temp=Leg_t1_2022_75[Leg_t1_2022_75['Nuance']=='NUP'][['id_bv','% Voix/Exp']]
BDV4_Scores_map = pd.merge(BDV_Map_75,Leg_t1_2022_75temp,how='inner',left_on=['id_bv'],right_on=['id_bv'])

In [ ]:
# First, ensure there are no non-numeric values in the column
BDV4_Scores_map['% Voix/Exp'] = pd.to_numeric(BDV4_Scores_map['% Voix/Exp'], errors='coerce')

# Now, convert the column to integer
BDV4_Scores_map['% Voix/Exp'] = BDV4_Scores_map['% Voix/Exp'].astype('Int64')  # Using 'Int64' to allow for NaNs

In [ ]:
BDV4_Scores_map.explore(
    m=m,
    column='% Voix/Exp', # make choropleth based on "BoroName" column
    tooltip=['% Voix/Exp',"id_bv"], # show "BoroName" value in tooltip (on hover)
    popup=True, # show all values in popup (on click)
    tiles="OpenStreetMap", # use "CartoDB positron" tiles
    cmap="Reds", # use "Set1" matplotlib colormap
    name="Score Natalie législatives 2022 (votes exprimés)",
    legend_kwds={'colorbar' : True, 'scale': True},
    show=False# name of the layer in the map
    #style_kwds=dict(color="black") # use black outline
)

folium.LayerControl(collapsed=False).add_to(m)  # use folium to add layer control

m.save('gerg.html')